In [ ]:
import sys
sys.path.append('/Users/abc/Sites/Agentic/Notebooks/venv/lib/python3.12/site-packages')

In [7]:
import numpy
import pmdarima

print("NumPy Version:", numpy.__version__)
print("pmdarima Version:", pmdarima.__version__)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [5]:

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pmdarima import auto_arima
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph
from datetime import datetime, timedelta

# Step 1: Define Nodes in LangGraph
class MarketAnalysisState:
    def __init__(self):
        self.data = None
        self.predictions = None
        self.report = None

# Node 1: Data Ingestion
def fetch_financial_data(state: MarketAnalysisState, ticker: str = "AAPL", period: str = "6mo"):
    print("Fetching data for:", ticker)
    df = yf.download(ticker, period=period)
    state.data = df['Close']
    return state

# Node 2: Market Analysis & Prediction (Auto ARIMA)
def market_analysis(state: MarketAnalysisState):
    print("Training Auto ARIMA model...")
    model = auto_arima(state.data, seasonal=False, suppress_warnings=True)
    forecast, conf_int = model.predict(n_periods=7, return_conf_int=True)
    
    state.predictions = {
        "dates": [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 8)],
        "forecast": forecast.tolist(),
        "confidence_interval": conf_int.tolist()
    }
    return state

# Node 3: Report Generation using LLM
def generate_report(state: MarketAnalysisState):
    llm = ChatOpenAI(model_name="gpt-4")
    prompt = f"""
    Analyze the following stock market predictions:
    Dates: {state.predictions['dates']}
    Forecasted Prices: {state.predictions['forecast']}
    
    Provide insights on expected trends, risks, and opportunities.
    """
    state.report = llm.predict(prompt)
    return state

# Node 4: Visualization
def plot_predictions(state: MarketAnalysisState):
    plt.figure(figsize=(10, 5))
    plt.plot(state.data.index, state.data, label="Historical Prices")
    plt.plot(state.predictions["dates"], state.predictions["forecast"], marker="o", linestyle="dashed", label="Predictions")
    plt.fill_between(state.predictions["dates"], np.array(state.predictions["confidence_interval"])[:, 0],
                     np.array(state.predictions["confidence_interval"])[:, 1], color='gray', alpha=0.3)
    plt.title("Stock Price Prediction (Next 7 Days)")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid()
    plt.show()
    return state

# Step 4: Build LangGraph Pipeline
graph = StateGraph(MarketAnalysisState)
graph.add_node("fetch_data", fetch_financial_data)
graph.add_node("predict", market_analysis)
graph.add_node("report", generate_report)
graph.add_node("visualize", plot_predictions)

graph.add_edge("fetch_data", "predict")
graph.add_edge("predict", "report")
graph.add_edge("report", "visualize")

graph.set_entry_point("fetch_data")
workflow = graph.compile()

# Step 5: Run the Workflow
state = workflow.invoke(MarketAnalysisState())
print("Generated Report:\n", state.report)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject